In [ ]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [ ]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,ModelDeployOp)
from google_cloud_pipeline_components.types import artifact_types

In [ ]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "assignment-1-399115"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2023_2056332"
# Specify your assignment
ASSIGNMENT = "Assignment_1"

In [ ]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, feature_dataset_name: str, label_dataset_name: str, merged_dataset: Output[Dataset]):
    '''Download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Get the client and bucket
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)

     # Download the feature dataset
    blob1 = bucket.blob(feature_dataset_name)
    blob1.download_to_filename(feature_dataset_name.path + ".csv")
    df_feature = pd.read_csv(feature_dataset_name.path + ".csv")
    
    
    # Download the label dataset
    blob2 = bucket.blob(label_dataset_name)
    blob2.download_to_filename(label_dataset_name.path + ".csv")
    df_label = pd.read_csv(feature_dataset_name.path + ".csv")
    
    
    # Merge the datasets
    merged_df = pd.merge(df_feature, df_label, on='Ind_ID')
    merged_df.to_csv(merged_dataset.path + ".csv", index=False, encoding='utf-8-sig')
    

    logging.info('Download & Merge all Data complete!')

In [ ]:
@dsl.component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.10.7-slim"
)
def clean_data(merged_dataset: Input[Dataset], cleaned_dataset: Output[Dataset]):
    '''Deletes irrelevant columns and removes NA's'''
    import pandas as pd
    import logging
    import sys
    import numpy

    # Sets the logging config
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 

    #Loads the merged dataset in
    df = pd.read_csv(merged_dataset.path, index_col=None)

    # Drops the columns: Birthday_count, Employed_days, Mobile_phone, Work_Phone, Phone, EMAIL_ID, Type_Occupation and Family_Members.
    cleandf = df.drop(columns= ["Birthday_count", "Employed_days", "Mobile_phone", "Work_Phone", "Phone", "EMAIL_ID", "Type_Occupation", "Family_Members"])

    # Save the cleaned dataset
    cleandf.to_csv(cleaned_dataset.path + ".csv", index=False, encoding='utf-8-sig')

In [ ]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def train_test_split(cleaned_dataset: Input[Dataset], dataset_train: Output[Dataset], dataset_test: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split as tts
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    alldata = pd.read_csv(cleaned_dataset.path, index_col=None)
    train, test = tts(alldata, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

### Smote toevoegen

In [ ]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_dataset_to_gcs(project_id: str, dataset_name: str, assignment: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(dataset_name)
     
   
    blob.upload_from_filename(dataset_name.path)   

In [ ]:
# pipeline component
@kfp.dsl.pipeline(
    name="diabetes-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, model_repo: str, dataset_feature_name: str, dataset_label_name: str):    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        feature_dataset_name= dataset_feature_name,
        label_dataset_name = dataset_label_name
    )
     
    clean_op = clean_data(merged_dataset =  di_op.outputs["merged_dataset"]) 
                                            # cleaned_dataset: Output[Dataset]) 
                                            
    train_test_split_op = train_test_split(cleaned_dataset = clean_op.outputs["cleaned_dataset"])
                                           # dataset_train: Output[Dataset], dataset_test: Output[Dataset])
        
    train_lr_op = train_lr(train_dataset = train_test_split_op.outputs["dataset_train"])
                                            # model: Output[Model])
    
    train_knn_op = train_knn(train_dataset = train_test_split_op.outputs["dataset_train"])
                                            # model: Output[Model]):
    
    algo_selection_op = compare_model(metrics_knn = train_knn_op.outputs["metrics_knn"], metrics_lr = train_lr_op.outputs["metrics_lr"]).after(train_lr_op, train_knn_op) 

    # defining the different conditions of algo selection                                 
    with dsl.If(algo_selection_op.output=="KNN"):
        predict_knn_op = predict_knn(model_knn = train_knn_op.outputs["model"],test_data = train_test_split_op.outputs["dataset_test"]
                                     )
        upload_model_knn_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=train_knn_op.outputs['model']
        ).after(predict_knn_op)    
        
    with dsl.If(algo_selection_op.output=="LR"):
        predict_lr_op = predict_lr(model_lr = train_lr_op.outputs["model"],test_data = train_test_split_op.outputs["dataset_test"])
        upload_model_knn_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=train_lr_op.outputs['model']
        ).after(predict_knn_op)  

In [ ]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='assignment1_pipeline.yaml')

In [ ]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="assignment_1",
    enable_caching=False,
    template_path="assignment1_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2023_2056332',  # the data_bucket name
        'model_repo':'models_de2023_2056332', # GCP model repo bucket name
        'dataset_feature_name' : 'Credit_card', # the feature dataset name
        'dataset_label_name': 'Credit_card_label' # the label dataset name
    }
)

job.run()